## Using the Quandle API

Quandle API is pretty awesome! There is a huge amount of data available here for free! However, the data isn't always the best quality. We were looking for daily recordings from present day back as far as possible and we had to snoop around in order to find data sets that contained fairly consistent recordings for a long period of time. This segments uses the api to pull a couple commodities and join them by date

In [1]:
QUANDLE_API_KEY = "2jWcwdZcAecTBtixj8Wz"

In [2]:
import requests
import time
import pandas as pd

In [3]:
gold = pd.read_csv("https://www.quandl.com/api/v3/datasets/LBMA/GOLD.csv?api_key=%s" % QUANDLE_API_KEY, 
                  index_col=0)

In [4]:
gold.head(3)

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM)
Date,,,,,,
2018-06-13,1294.40,1296.15,971.58,970.95,1101.92,1101.34
2018-06-12,1298.30,1298.65,968.27,972.30,1100.44,1101.90
2018-06-11,1296.05,1299.60,969.32,970.93,1099.57,1100.89


In [5]:
gold.tail(3)

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM)
Date,,,,,,
1968-01-04,35.14,NaN,14.603,NaN,NaN,NaN
1968-01-03,35.16,NaN,14.617,NaN,NaN,NaN
1968-01-02,35.18,NaN,14.641,NaN,NaN,NaN


In [6]:
gold.idxmin()

USD (AM)     1970-01-19
USD (PM)     1970-01-19
GBP (AM)     1970-01-19
GBP (PM)     1970-01-16
EURO (AM)    1999-08-05
EURO (PM)    1999-08-05
dtype: object

In [7]:
crude_oil = pd.read_csv("https://www.quandl.com/api/v3/datasets/CHRIS/CME_CL1.csv?api_key=%s" % QUANDLE_API_KEY, 
                       index_col=0)

In [8]:
crude_oil.idxmin()

Open                          1986-04-01
High                          1998-12-11
Low                           1986-04-01
Last                          1986-03-31
Change                        2017-10-17
Settle                        1986-03-31
Volume                        2012-04-06
Previous Day Open Interest    2000-02-11
dtype: object

In [9]:
gas = pd.read_csv("https://www.quandl.com/api/v3/datasets/FRED/GASALLCOVW.csv?api_key=%s" % QUANDLE_API_KEY,
                 index_col=0)

In [10]:
bit_cap = pd.read_csv("https://www.quandl.com/api/v3/datasets/BCHAIN/MKTCP.csv?api_key=%s" % QUANDLE_API_KEY, 
                     index_col=0)

In [11]:
futures_2 = pd.read_csv("https://www.quandl.com/api/v3/datasets/CHRIS/CME_TU2.csv?api_key=%s" % QUANDLE_API_KEY,
                       index_col=0)

In [12]:
futures_2.idxmin()

Open                          2007-10-15
High                          2007-10-15
Low                           2007-10-15
Last                          2007-10-12
Change                        2014-12-08
Settle                        2007-10-12
Volume                        2018-04-09
Previous Day Open Interest    2018-04-10
dtype: object

In [13]:
futures_10 = pd.read_csv("https://www.quandl.com/api/v3/datasets/CHRIS/CME_TY2.csv?api_key=%s" % QUANDLE_API_KEY,
                        index_col=0)


In [14]:
predictors = gold[["USD (AM)"]].merge(
    crude_oil[["Settle"]], 
    how="outer", 
    right_index = True, 
    left_index = True
).rename(index=str, columns={"USD (AM)": "gold", "Settle": "crude_oil"})

# predictors.head()

In [15]:
predictors = predictors.merge(
    gas, 
    how="outer", 
    right_index = True, 
    left_index = True
).rename(index=str, columns={"Value": "gas"})

# predictors.head()

In [16]:
predictors = predictors.merge(
    bit_cap, 
    how="outer", 
    right_index = True, 
    left_index = True
).rename(index=str, columns={"Value": "bit_cap"})

# predictors.head()

In [17]:
predictors = predictors.merge(
    futures_2[["Settle"]], 
    how="outer", 
    right_index = True, 
    left_index = True
).rename(index=str, columns={"Settle": "futures_2"})

In [18]:
predictors = predictors.merge(
    futures_10[["Settle"]], 
    how="outer", 
    right_index = True, 
    left_index = True
).rename(index=str, columns={"Settle": "futures_10"})

In [19]:
predictors = predictors[predictors.index > "2008"]
predictors.head()

,gold,crude_oil,gas,bit_cap,futures_2,futures_10
Date,,,,,,
2008-01-02,840.75,99.62,NaN,NaN,105.507812,113.765625
2008-01-03,865.35,99.18,NaN,NaN,105.593750,113.718750
2008-01-04,858.75,97.91,NaN,NaN,105.796875,114.375000
2008-01-07,857.50,95.09,3.135,NaN,105.718750,114.546875
2008-01-08,873.25,96.33,NaN,NaN,105.718750,114.546875


In [20]:
import pickle
pickle.dump(predictors, open("predictors.pkl", "wb"))

## Using the S&P 500 data set from Kaggel Dataset

This segment grabs a relvent feature from the CSV for all tickers. We didn't 

In [21]:
all_stocks = pd.read_csv("data/all_stocks_5yr.csv")
all_stocks.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [22]:
daily_stock = all_stocks[["date", "Name", "close"]].set_index(["date", "Name"]).unstack(level=1)
daily_stock = daily_stock[["close"][0]]

In [23]:
daily_stock.head()

Name,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-02-08,45.08,14.75,78.90,67.8542,36.25,46.89,34.41,73.31,39.12,45.70,...,28.24,37.51,88.61,42.87,31.84,27.09,65.30,75.85,24.14,33.05
2013-02-11,44.60,14.46,78.39,68.5614,35.85,46.76,34.26,73.07,38.64,46.08,...,28.31,37.46,88.28,42.84,31.96,27.46,64.55,75.65,24.21,33.26
2013-02-12,44.62,14.27,78.60,66.8428,35.42,46.96,34.30,73.37,38.89,46.27,...,28.41,37.58,88.46,42.87,31.84,27.95,64.75,75.44,24.49,33.74
2013-02-13,44.75,14.66,78.97,66.7156,35.27,46.64,34.46,73.56,38.81,46.26,...,28.42,37.80,88.67,43.08,32.00,28.26,64.41,76.00,24.74,33.55
2013-02-14,44.58,13.99,78.84,66.6556,36.57,46.77,34.70,73.13,38.61,46.54,...,28.22,38.44,88.52,42.91,32.12,28.47,63.89,76.34,24.63,33.27


In [24]:
import pickle
pickle.dump(daily_stock, open("daily_stock.pkl", "wb"))

## Trend Data From Google
Wanted to get something working which could scrape the API for consistent readings over a long period of time. THIS WAS A STRUGGLE. 

In [25]:
import requests
import json
import pandas as pd
g_trends = requests.get("https://trends.google.com/trends/explore?date=now%201-H&geo=US&q=stocks")
g_trends.text[-300:]

';height:54px;width:150px}</style><div id="af-error-container"><a href=//www.google.com><span id=logo aria-label=Google></span></a><p><b>429.</b> <ins>That’s an error.</ins><p>We\'re sorry, but you have sent too many requests to us recently. Please try again later. <ins>That’s all we know.</ins></div>'

In [26]:
cookies = dict(filter(
            lambda i: i[0] == 'NID',
            requests.get('https://trends.google.com').cookies.items()
        ))
cookies

{'NID': '132=z_kuFfT5VudHVvhO02hkcAMctmayQXmSDqfDO8llzfDB2vwKWrC8flXSYsw2BpMo10c0_CDA2GIL8YL5qtF1Z5pED6hMxkmFCm6SiR9PFjnBfLtwdjqPoIqVFanWM3oT'}

In [27]:
response = requests.get("https://trends.google.com/trends/explore?date=now%201-H&geo=US&q=stocks",
                       cookies=cookies)
content = response.text
response.text[:300]

'<!doctype html><html itemscope itemtype="https://schema.org/WebPage" dir="ltr"><head><meta name="google-site-verification" content="-uo2JByp3-hxDA1ZgvM3dP8BE1_qDDddCm_st_w41P8"/><title>stocks - Google Trends</title><link rel="icon" href="https://ssl.gstatic.com/trends_nrtr/1457_RC03/favicon.ico" typ'

In [28]:
stocks = "https://trends.google.com/trends/api/widgetdata/multiline/csv?req=%7B%22time%22%3A%222018-06-06T17%5C%5C%3A48%5C%5C%3A15%202018-06-06T18%5C%5C%3A48%5C%5C%3A15%22%2C%22resolution%22%3A%22MINUTE%22%2C%22locale%22%3A%22en-US%22%2C%22comparisonItem%22%3A%5B%7B%22geo%22%3A%7B%22country%22%3A%22US%22%7D%2C%22complexKeywordsRestriction%22%3A%7B%22keyword%22%3A%5B%7B%22type%22%3A%22BROAD%22%2C%22value%22%3A%22stocks%22%7D%5D%7D%7D%5D%2C%22requestOptions%22%3A%7B%22property%22%3A%22%22%2C%22backend%22%3A%22CM%22%2C%22category%22%3A0%7D%7D&token=APP6_UEAAAAAWxl97_ylQR5Jh6KSFpspCUYtxNi3nSbR&tz=420"

In [29]:
# pd.read_csv(stocks).head()

In [30]:
import urllib.parse
query = urllib.parse.parse_qs(
    urllib.parse.urlparse(stocks).query
)

query

{'req': ['{"time":"2018-06-06T17\\\\:48\\\\:15 2018-06-06T18\\\\:48\\\\:15","resolution":"MINUTE","locale":"en-US","comparisonItem":[{"geo":{"country":"US"},"complexKeywordsRestriction":{"keyword":[{"type":"BROAD","value":"stocks"}]}}],"requestOptions":{"property":"","backend":"CM","category":0}}'],
 'token': ['APP6_UEAAAAAWxl97_ylQR5Jh6KSFpspCUYtxNi3nSbR'],
 'tz': ['420']}

In [31]:
json_req = json.loads(query['req'][0])
json_req

{'time': '2018-06-06T17\\:48\\:15 2018-06-06T18\\:48\\:15',
 'resolution': 'MINUTE',
 'locale': 'en-US',
 'comparisonItem': [{'geo': {'country': 'US'},
   'complexKeywordsRestriction': {'keyword': [{'type': 'BROAD',
      'value': 'stocks'}]}}],
 'requestOptions': {'property': '', 'backend': 'CM', 'category': 0}}

In [32]:
json_req['comparisonItem'][0]['complexKeywordsRestriction']['keyword'][0]['value'] = "banana"

json_req

{'time': '2018-06-06T17\\:48\\:15 2018-06-06T18\\:48\\:15',
 'resolution': 'MINUTE',
 'locale': 'en-US',
 'comparisonItem': [{'geo': {'country': 'US'},
   'complexKeywordsRestriction': {'keyword': [{'type': 'BROAD',
      'value': 'banana'}]}}],
 'requestOptions': {'property': '', 'backend': 'CM', 'category': 0}}

In [33]:
query["req"][0] = json_req
query

{'req': [{'time': '2018-06-06T17\\:48\\:15 2018-06-06T18\\:48\\:15',
   'resolution': 'MINUTE',
   'locale': 'en-US',
   'comparisonItem': [{'geo': {'country': 'US'},
     'complexKeywordsRestriction': {'keyword': [{'type': 'BROAD',
        'value': 'banana'}]}}],
   'requestOptions': {'property': '', 'backend': 'CM', 'category': 0}}],
 'token': ['APP6_UEAAAAAWxl97_ylQR5Jh6KSFpspCUYtxNi3nSbR'],
 'tz': ['420']}

In [34]:
new_query = urllib.parse.urlencode(query)
new_query

'req=%5B%7B%27time%27%3A+%272018-06-06T17%5C%5C%3A48%5C%5C%3A15+2018-06-06T18%5C%5C%3A48%5C%5C%3A15%27%2C+%27resolution%27%3A+%27MINUTE%27%2C+%27locale%27%3A+%27en-US%27%2C+%27comparisonItem%27%3A+%5B%7B%27geo%27%3A+%7B%27country%27%3A+%27US%27%7D%2C+%27complexKeywordsRestriction%27%3A+%7B%27keyword%27%3A+%5B%7B%27type%27%3A+%27BROAD%27%2C+%27value%27%3A+%27banana%27%7D%5D%7D%7D%5D%2C+%27requestOptions%27%3A+%7B%27property%27%3A+%27%27%2C+%27backend%27%3A+%27CM%27%2C+%27category%27%3A+0%7D%7D%5D&token=%5B%27APP6_UEAAAAAWxl97_ylQR5Jh6KSFpspCUYtxNi3nSbR%27%5D&tz=%5B%27420%27%5D'

In the page the file download is controlled by this function:

```
d.exportAsCSV = function(a, b, c) {
    a = encodeURIComponent(JSON.stringify(a));
    c = "/trends/api/widgetdata" + ("/" + this.convertWidgetTypeToApiName_(c) + "/csv");
    c = c + ("?req=" + a + "&token=" + b) + ("&tz=" + this.configService_.userTimezoneOffset);
    this.downloadCsvFile_(c)
}
```

So we need to figure out how to get a proper token! Grrr...

In [35]:
stocks = "https://trends.google.com/trends/api/widgetdata/multiline/csv?req=%7B%22time%22%3A%222018-06-06T18%5C%5C%3A58%5C%5C%3A33%202018-06-06T19%5C%5C%3A58%5C%5C%3A33%22%2C%22resolution%22%3A%22MINUTE%22%2C%22locale%22%3A%22en-US%22%2C%22comparisonItem%22%3A%5B%7B%22geo%22%3A%7B%22country%22%3A%22US%22%7D%2C%22complexKeywordsRestriction%22%3A%7B%22keyword%22%3A%5B%7B%22type%22%3A%22BROAD%22%2C%22value%22%3A%22stocks%22%7D%5D%7D%7D%5D%2C%22requestOptions%22%3A%7B%22property%22%3A%22%22%2C%22backend%22%3A%22CM%22%2C%22category%22%3A0%7D%7D&token=APP6_UEAAAAAWxmOaSwlhHrJ0qdDSaXTYRtDy0PAG0sk&tz=420"

In [36]:
import re
def download_from_topic_token(topic, token):
    DOWNLOAD_URL = "https://trends.google.com/trends/api/widgetdata/multiline/csv?req=%7B%22time%22%3A%222018-06-06T18%5C%5C%3A59%5C%5C%3A06%202018-06-06T19%5C%5C%3A59%5C%5C%3A06%22%2C%22resolution%22%3A%22MINUTE%22%2C%22locale%22%3A%22en-US%22%2C%22comparisonItem%22%3A%5B%7B%22geo%22%3A%7B%22country%22%3A%22US%22%7D%2C%22complexKeywordsRestriction%22%3A%7B%22keyword%22%3A%5B%7B%22type%22%3A%22BROAD%22%2C%22value%22%3A%22" + topic + "%22%7D%5D%7D%7D%5D%2C%22requestOptions%22%3A%7B%22property%22%3A%22%22%2C%22backend%22%3A%22CM%22%2C%22category%22%3A0%7D%7D&token=" + token +"&tz=420"
    return DOWNLOAD_URL

def get_token(topic):
    response = requests.get(
        "https://trends.google.com/trends/api/explore?hl=en-US&tz=420&req=%7B%22comparisonItem%22:%5B%7B%22keyword%22:%22" + topic + "%22,%22geo%22:%22US%22,%22time%22:%22now+1-H%22%7D%5D,%22category%22:0,%22property%22:%22%22%7D&tz=420", cookies=cookies)
    finding_token = response.text.split("token")[1]

    t = re.sub('[^0-9a-zA-Z_]+', ' ', finding_token)
    gotcha = t.split()[0]
    return gotcha

In [37]:
requests.get(download_from_topic_token("stocks", get_token("stocks"))).text[-300:]

'ize:100% 100%}}#logo{display:inline-block;height:54px;width:150px}</style><div id="af-error-container"><a href=//www.google.com><span id=logo aria-label=Google></span></a><p><b>401.</b> <ins>That’s an error.</ins><p>The requested URL was not found on this server. <ins>That’s all we know.</ins></div>'

In [38]:
x = download_from_topic_token("banana", get_token("banana"))
x

'https://trends.google.com/trends/api/widgetdata/multiline/csv?req=%7B%22time%22%3A%222018-06-06T18%5C%5C%3A59%5C%5C%3A06%202018-06-06T19%5C%5C%3A59%5C%5C%3A06%22%2C%22resolution%22%3A%22MINUTE%22%2C%22locale%22%3A%22en-US%22%2C%22comparisonItem%22%3A%5B%7B%22geo%22%3A%7B%22country%22%3A%22US%22%7D%2C%22complexKeywordsRestriction%22%3A%7B%22keyword%22%3A%5B%7B%22type%22%3A%22BROAD%22%2C%22value%22%3A%22banana%22%7D%5D%7D%7D%5D%2C%22requestOptions%22%3A%7B%22property%22%3A%22%22%2C%22backend%22%3A%22CM%22%2C%22category%22%3A0%7D%7D&token=APP6_UEAAAAAWyQhF57720JF9baM6uI&tz=420'

In [39]:
requests.get(x, cookies=cookies).text[-200:]

'r"><a href=//www.google.com><span id=logo aria-label=Google></span></a><p><b>401.</b> <ins>That’s an error.</ins><p>The requested URL was not found on this server. <ins>That’s all we know.</ins></div>'

In [40]:
urllib.parse.parse_qs(
    urllib.parse.urlparse(stocks).query
)

{'req': ['{"time":"2018-06-06T18\\\\:58\\\\:33 2018-06-06T19\\\\:58\\\\:33","resolution":"MINUTE","locale":"en-US","comparisonItem":[{"geo":{"country":"US"},"complexKeywordsRestriction":{"keyword":[{"type":"BROAD","value":"stocks"}]}}],"requestOptions":{"property":"","backend":"CM","category":0}}'],
 'token': ['APP6_UEAAAAAWxmOaSwlhHrJ0qdDSaXTYRtDy0PAG0sk'],
 'tz': ['420']}

In [41]:
cookies

{'NID': '132=z_kuFfT5VudHVvhO02hkcAMctmayQXmSDqfDO8llzfDB2vwKWrC8flXSYsw2BpMo10c0_CDA2GIL8YL5qtF1Z5pED6hMxkmFCm6SiR9PFjnBfLtwdjqPoIqVFanWM3oT'}

## I am giving up on google trends. urg. 
 Too much work for too little value add.

### Dang! After all that we still cant programatically download a csv. 

Something seems wrong with the way we are generating a token. The plan was to write a function to download the daily data for our time frame (over 5 years). However instead we are just going to use the less granular data and call it a day.


In [42]:
sp_google = pd.read_csv("data/S&P 500.csv", skiprows=1, index_col=0)

predictors = predictors.merge(
    sp_google, 
    how="outer", 
    right_index = True, 
    left_index = True
).rename(index=str, columns={"S&P 500 Index: (United States)": "sp_interest"})

predictors.sp_interest = predictors.sp_interest.fillna(method='ffill')

In [43]:
debt_google = pd.read_csv("data/debt.csv", skiprows=1, index_col=0)
predictors = predictors.merge(
    debt_google, 
    how="outer", 
    right_index = True, 
    left_index = True
).rename(index=str, columns={"debt: (United States)": "debt_interest"})

predictors.debt_interest = predictors.debt_interest.fillna(method='ffill')

import pickle
pickle.dump(predictors, open("predictors.pkl", "wb"))

predictors.tail()

,gold,crude_oil,gas,bit_cap,futures_2,futures_10,sp_interest,debt_interest
2018-06-09,NaN,NaN,NaN,1.301751e+11,NaN,NaN,35.0,36.0
2018-06-10,NaN,NaN,NaN,1.292363e+11,NaN,NaN,27.0,34.0
2018-06-11,1296.05,66.10,2.883,1.157925e+11,105.859375,119.43750,27.0,34.0
2018-06-12,1298.30,66.36,NaN,1.174925e+11,105.851562,119.40625,27.0,34.0
2018-06-13,1294.40,66.64,NaN,1.116841e+11,105.765625,119.18750,27.0,34.0


### [News Sentiment](http://127.0.0.1:8000/NewsDataCleaning.slides.html#/)

### [Back to our main presentation](http://127.0.0.1:8000/Presentation.slides.html#/3)

In [44]:
!jupyter nbconvert DataCleaning.ipynb --to slides --post serve

[NbConvertApp] Converting notebook DataCleaning.ipynb to slides
[NbConvertApp] Writing 274737 bytes to DataCleaning.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Serving your slides at http://127.0.0.1:8000/DataCleaning.slides.html
Use Control-C to stop this server
^C

Interrupted
